In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd

In [2]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [3]:
tf.__version__

'1.13.1'

In [4]:
tf.keras.__version__

'2.2.4-tf'

In [7]:
tweets = pd.read_csv('../data_master/resources/democrats.csv')
tweets

,Political Affiliation,tweet,Politician,Word Count
0,Democrats,I am so incredibly proud of @CabanForQueens - ...,Alexandria Ocasio-Cortez,22
1,Democrats,😎 https://t.co/p20O6WLNSe,Alexandria Ocasio-Cortez,2
2,Democrats,Wayfair workers couldn’t stomach they were mak...,Alexandria Ocasio-Cortez,20
3,Democrats,It’s wild to think that Third Way has gotten a...,Alexandria Ocasio-Cortez,24
4,Democrats,Queens residents: Today is the day we vote for...,Alexandria Ocasio-Cortez,23
5,Democrats,Reminder: Caging kids is a for-profit enterpri...,Alexandria Ocasio-Cortez,19
6,Democrats,"I’ll never forget this, because it was the mom...",Alexandria Ocasio-Cortez,25
7,Democrats,"Yes, the treatment is bad - but the one thing ...",Alexandria Ocasio-Cortez,24
8,Democrats,"If you doubt it, here’s the original tweet. Th...",Alexandria Ocasio-Cortez,20
9,Democrats,Reminder: the member who directly + explicitly...,Alexandria Ocasio-Cortez,19


In [9]:
import imdb

In [8]:
tweets.tweet

0        I am so incredibly proud of @CabanForQueens - ...
1                                😎 https://t.co/p20O6WLNSe
2        Wayfair workers couldn’t stomach they were mak...
3        It’s wild to think that Third Way has gotten a...
4        Queens residents: Today is the day we vote for...
5        Reminder: Caging kids is a for-profit enterpri...
6        I’ll never forget this, because it was the mom...
7        Yes, the treatment is bad - but the one thing ...
8        If you doubt it, here’s the original tweet. Th...
9        Reminder: the member who directly + explicitly...
10       The student loan crisis is keeping people from...
11       Student loans are a scam.\n\nThat’s why I join...
12       We’re just a couple of girls, standing in fron...
13       Remember when the Bush administration bullied ...
14       I have absolutely zero patience for completely...
15       The last time you went on this trip it was rep...
16       Hey @Liz_Cheney, you’re the GOP Conference Cha.

In [11]:
tweets_to_list = tweets.tweet.tolist()
print(tweets_to_list[0:5])
tweets_to_list


['I am so incredibly proud of @CabanForQueens - and EVERY single person who showed up for this election today.\n\nNo ma… https://t.co/cyZfvag3de', '😎 https://t.co/p20O6WLNSe', 'Wayfair workers couldn’t stomach they were making beds to cage children.\n\nThey asked the company to stop. CEO said… https://t.co/bEG5kaGL6x', 'It’s wild to think that Third Way has gotten along with its “sensible Dem” charade as long as it has.\n\nI’ve met Tru… https://t.co/xDwRGYAu1g', 'Queens residents: Today is the day we vote for DA! 🌞🗳 \n\nCommunities win when we turn out in high numbers. Let’s get… https://t.co/R2EmnOCx2p']


['I am so incredibly proud of @CabanForQueens - and EVERY single person who showed up for this election today.\n\nNo ma… https://t.co/cyZfvag3de',
 '😎 https://t.co/p20O6WLNSe',
 'Wayfair workers couldn’t stomach they were making beds to cage children.\n\nThey asked the company to stop. CEO said… https://t.co/bEG5kaGL6x',
 'It’s wild to think that Third Way has gotten along with its “sensible Dem” charade as long as it has.\n\nI’ve met Tru… https://t.co/xDwRGYAu1g',
 'Queens residents: Today is the day we vote for DA! 🌞🗳 \n\nCommunities win when we turn out in high numbers. Let’s get… https://t.co/R2EmnOCx2p',
 'Reminder: Caging kids is a for-profit enterprise, &amp; the people who profit off camps finance the political campaigns… https://t.co/GudAoqYleD',
 'I’ll never forget this, because it was the moment I saw with my own eyes that the America I love was becoming a nat… https://t.co/fbbgzGxfZN',
 'Yes, the treatment is bad - but the one thing I’ve learned about the right is that they

In [12]:
x_train_text, y_train = tweets_to_list(train=True)
x_test_text, y_test = tweets_to_list(train=False)

TypeError: 'list' object is not callable

Starting with this dataframe
    one  two
a  1.0  1.0
b  2.0  2.0
c  3.0  3.0
d  NaN  4.0
The first column is a <class 'pandas.core.series.Series'> 
consisting of
 a    1.0
b    2.0
c    3.0
d    NaN
Name: one, dtype: float64
dfToList is [1.0, 2.0, 3.0, nan] and it's a <class 'list'>
dfList is   [1.0, 2.0, 3.0, nan] and it's a <class 'list'>
dfValues is [ 1.  2.  3. nan] and it's a <class 'numpy.ndarray'>
